In [1]:
import pandas as pd
!conda install -c conda-forge/label/cf201901 wikipedia
import wikipedia as wp


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - wikipedia


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge/label/cf201901
    ca-certificates-2018.11.29 |       ha4d7672_0         143 KB  conda-forge/label/cf201901
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge/label/cf201901
    wikipedia-1.4.0            |           py35_0          20 KB  conda-forge/label/cf201901
    ------------------------------------------------------------
                                           Total:         3.4 MB

The following NEW packages will be INSTALLED:

    wikipedia:       1.4.0-py35_0      conda-forge/label/cf201901

The following packages will be UPDATED:

    certifi:         2018.8.24-py35_1                

In [2]:
html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
Toronto_df = pd.read_html(html)[0]
Toronto_df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [3]:
Toronto_df.columns=Toronto_df.iloc[0]
Toronto_df.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [4]:
Toronto_df=Toronto_df.reindex(Toronto_df.index.drop(0))
Toronto_df.head()
Toronto_df.shape

(288, 3)

In [5]:
Toronto_database=Toronto_df[Toronto_df['Borough']!='Not assigned']
Toronto_database=Toronto_database.reset_index()
Toronto_database=Toronto_database.drop('index',1)
Toronto_database.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [6]:
Toronto_database1=Toronto_database[Toronto_database['Neighbourhood']=='Not assigned']
Toronto_database1=Toronto_database1.drop('Neighbourhood',1)
Toronto_database1['Neighbourhood']=Toronto_database1['Borough']
Toronto_database1

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Queen's Park


In [7]:
Toronto_database=Toronto_df[Toronto_df['Neighbourhood']!='Not assigned']
Toronto_database=Toronto_database.append(Toronto_database1)
Toronto_database.shape

(211, 3)

In [18]:

TDF=Toronto_database.groupby('Postcode').agg({
                'Borough': lambda x: x.unique(),
                'Neighbourhood': lambda x: ', '.join(x)
    })
TDF

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


In [19]:
TDF.shape

(103, 2)